# 0. Pip Installations

In [97]:
# Install all requirements

packages_to_install = ['opendatasets', 'polars', 'nltk', 'wordcloud', 'stop_words', 'torch', 'transformers']
def install_packages(pckg):
    try:
       __import__(pckg)
       print("Successfully imported", pckg)
    except ModuleNotFoundError:
        !pip install {pckg}
        print("Successfully installed", pckg)

for el in packages_to_install:
    install_packages(el)

Successfully imported opendatasets
Successfully imported polars
Successfully imported nltk
Successfully imported wordcloud
Successfully imported stop_words
Successfully imported torch
Successfully imported transformers


In [98]:
import torch
# specify GPU
device = torch.device("cuda")

# 1. Gather data

In [99]:
import opendatasets as od
import polars as pl

od.download("https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset")
file =('sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv')

# Enter kaggle username and api key to download dataset. Api key can be found by following this link:
# https://www.kaggle.com/settings/account

df = pl.read_csv(file,separator=',', encoding='ISO-8859-1')

Skipping, found downloaded files in ".\sentiment-analysis-dataset" (use force=True to force download)


In [100]:
df.head()

polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
i64,i64,str,str,str,str
0,1467810672,"""Mon Apr 06 22:…","""NO_QUERY""","""scotthamilton""","""is upset that …"
0,1467810917,"""Mon Apr 06 22:…","""NO_QUERY""","""mattycus""","""@Kenichan I di…"
0,1467811184,"""Mon Apr 06 22:…","""NO_QUERY""","""ElleCTF""","""my whole body …"
0,1467811193,"""Mon Apr 06 22:…","""NO_QUERY""","""Karoli""","""@nationwidecla…"
0,1467811372,"""Mon Apr 06 22:…","""NO_QUERY""","""joy_wolf""","""@Kwesidei not …"


# 2. Data Preprocessing

## 2.1 Data cleaning

In [101]:
df.null_count()
# No null data

# Rename columns
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']

# Drop columns that are not needed
df = df.select(['sentiment', 'text'])

df.head()

sentiment,text
i64,str
0,"""is upset that …"
0,"""@Kenichan I di…"
0,"""my whole body …"
0,"""@nationwidecla…"
0,"""@Kwesidei not …"


## 2.2 Remove Urls

In [102]:
import re

df = df.with_columns(pl.col('text').map_elements(lambda x: re.sub(r"http\S+","", x)))

## 2.3 Remove punctuations and special characters 

In [103]:
import string
df = df.with_columns(pl.col('text').map_elements(lambda x: x.translate(str.maketrans('', '', string.punctuation))))

## 2.4 Lowercase all text

In [104]:
df = df.with_columns(pl.col('text').str.to_lowercase())

## 2.5 Tokenization

In [105]:
df = df.with_columns(pl.col('text')
                     .str.split(by=" ")
                    .alias('tokenized_text'))
cleaned_df = df.drop('text')

## 2.5 Stemming

In [ ]:
from nltk.stem import SnowballStemmer
from stop_words import get_stop_words

stemmer = SnowballStemmer('english')
stemmed_df = cleaned_df.with_columns(pl.col('tokenized_text').map_elements(lambda x: [stemmer.stem(word) for word in x if (word and word not in get_stop_words('english'))]).alias('stemmed_text'))

In [ ]:
stemmed_df = stemmed_df.drop('tokenized_text')


# 3. Data Exploration

In [ ]:
stemmed_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Convert polarity to sentiment
# The polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

analysis_df = stemmed_df.clone()

mapping = {0: 'negative', 2: 'neutral', 4: 'positive'}

analysis_df = analysis_df.with_columns(pl.col('sentiment').map_dict(mapping).alias('sentiment'))

sentiment_count =analysis_df.select(pl.col('sentiment').value_counts(sort=True)).unnest('sentiment')
plt.barh(y=sentiment_count['sentiment'], width=sentiment_count['counts'])
plt.show()

This indicates more tweets with a negative sentiments than positive sentiments. No tweets could be found with a neutral sentiment.

In [ ]:
# Frequency of tokenized words
from collections import Counter
import numpy as np

def count_word_freq(column):
    return Counter([item for sublist in column.to_list() for item in sublist])

freq = count_word_freq(stemmed_df.get_column('stemmed_text'))
freq_negative_sentiment = count_word_freq(stemmed_df.filter(pl.col('sentiment') == 'negative').get_column('stemmed_text'))
freq_positive_sentiment = count_word_freq(stemmed_df.filter(pl.col('sentiment') == 'positive').get_column('stemmed_text'))

In [ ]:
# Visualize top 50 frequently used words
word, frequency = zip(*freq.most_common(n=50))
indices = np.arange(len(freq.most_common(n=50)))
plt.bar(indices, frequency, color='b')
plt.xticks(indices, word, rotation='vertical')
plt.title('Top 50 frequently used words')
plt.tight_layout()
plt.show()

In [ ]:
# Create wordcloud
import wordcloud

fig, (ax1, ax2) = plt.subplots(1, 2)

pos_cloud = wordcloud.WordCloud(max_words=150, random_state=30, collocations=True)
neg_cloud = wordcloud.WordCloud(max_words=150, random_state=30, collocations=True)
negative_sentiment_cloud = neg_cloud.generate_from_frequencies(dict(freq_negative_sentiment.most_common()))
positive_sentiment_cloud = pos_cloud.generate_from_frequencies(dict(freq_positive_sentiment.most_common()))

ax1.imshow(negative_sentiment_cloud, interpolation='bilinear')
ax1.set_title('Negative Sentiment Words')
ax1.axis('off')
ax2.imshow(positive_sentiment_cloud, interpolation='bilinear')
ax2.set_title('Positive Sentiment Words')
ax2.axis('off')

fig.tight_layout()
fig.set_size_inches(15,15)

# 4. Text Vectorization

In [ ]:
# Create model for word vectorization

from gensim.models import FastText

X = stemmed_df.select('stemmed_text')
y = stemmed_df.select('sentiment')



def train_model():
    vector_size = 100

    new_fasttext_model = FastText(vector_size= vector_size, window= 3, min_count= 1, sg=1)
    new_fasttext_model.build_vocab(X.to_series().to_list())
    new_fasttext_model.train(X.to_series().to_list(), total_examples=new_fasttext_model.corpus_count, epochs=5)

    new_fasttext_model.save("fasttext.model")

    return new_fasttext_model

def get_fasttext_model():
    try:
        pretrained_model = FastText.load("fasttext.model")
        print("Successfully loaded pretrained model")
        return pretrained_model
    except AttributeError:
        print("Training new model")
        return train_model()

fasttext_model = get_fasttext_model()

In [ ]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.to_series().to_list(), y.to_series().to_list(), test_size=0.2, random_state=42)


X_train_vectors = []
for sentence in X_train:
    sentence_vectors = []
    for token in sentence:
        try:
            word_vector = fasttext_model.wv[token]
            sentence_vectors.append(word_vector)
        except KeyError:
            # Handle out-of-vocabulary words here, e.g., replace with a placeholder vector
            pass

    if len(sentence_vectors) > 0:
        # Calculate the average of word vectors for the sentence
        sentence_vector = np.mean(sentence_vectors, axis=0)
        X_train_vectors.append(sentence_vector)
    else:
        pass

X_test_vectors = []
for sentence in X_test:
    sentence_vectors = []
    for token in sentence:
        try:
            word_vector = fasttext_model.wv[token]
            sentence_vectors.append(word_vector)
        except KeyError:
            pass

    if len(sentence_vectors) > 0:
        sentence_vector = np.mean(sentence_vectors, axis=0)
        X_test_vectors.append(sentence_vector)
    else:
        pass

print(X_train_vectors[1])
print(X_test_vectors[1])


## 5. Model (BERT)

In [ ]:
from transformers import BertForSequenceClassification

MODEL_NAME = 'bert-base-uncased'

model = BertForSequenceClassification.from_pretrained(MODEL_NAME)

X_train_tensors = torch.tensor(X_train_vectors)
X_test_tensors = torch.tensor(X_test_vectors)
y_train_tensors = torch.tensor(y_train)
y_test_tensors = torch.tensor(y_test)